# Numerical model of a straight arterial section

The model equation that describes flow through the basement membrane requires the position $R_i(z,t)$ and pressure $p(z,t)$ as inputs, which shall be obtained from a numerical model of the human middle cerebral artery (MCA). This model was used to demonstrate how arterial pulsations due to the heart beat are not strong enough to drive intramural peri-arterial drainage (IPAD) [1]. To model the MCA we use the Python library [VaMpy](https://github.com/akdiem/vampy) [2], which implements the 1D blood flow equations as derived by Olufsen [3].

Start by importing VaMpy (and some other libraries that will be useful)

In [1]:
from vampy import *
import numpy as np
from scipy.interpolate import interp1d

To set up the artery geometry we need to define a few parameters. We will also define characteristic parameters such that the calculations are performed dimensionless

In [2]:
# Characteristic parameters
R = 1 # cm, radius
Q = 10 # cm*3/s, flow rate
RHO = 1.06 # g/cm**3, blood density
NU = 0.046 # cm**2/s, blood viscosity
RE = Q / (NU*R) # Reynolds number
KEL = RHO * Q**2 / R**4 # g / (cm * s**2), elasticity parameters
RES = Q * RHO / R**4
COM = R**7 / (RHO * Q**2)

# Parameters
Ru = np.array([0.14, 0.13, 0.13])/R # upstream radii per vessel
Rd = np.array([0.14, 0.13, 0.13])/R # downstream radii per vessel
Lam = np.array([50, 50, 49.5]) # length factor
k1 = 2.0e7 / KEL # elasticity parameter
k2 = -22.53 * R # elasticity parameter
k3 = 8.65e5 / KEL # elasticity parameter
K = np.array([k1, k2, k3])
p0 = (80 * 1333.22365) / KEL # zero transmural pressure
nu = NU*R/Q # dimensionless viscosity
depth = 2 # artery network depth
N = 50 # number of output timesteps
dx = 0.05 / R # spatial step size
dt = 5e-6 * Q/R**3 # time step size
T = 0.85 * Q/R**3 # length of one cycle
tc = 4 # number of cycles

The simulation is setup as an ArteryNetwork object

In [3]:
an = ArteryNetwork(Ru, Rd, Lam, K, RHO, nu, p0, depth, N, RE)
an.mesh(dx)
an.set_time(dt, T, tc)
an.initial_conditions(0.0)

To solve the equations we need to define inlet and outlet boundary conditions. At the inlet flow rate is prescribed while the outlet is a 3-element Windkessel

In [4]:
def mca_inlet(Ru, Q, T):
    V = np.loadtxt("./mca_vampy/mca_inlet.csv", delimiter=',')
    t = [(elem/V[-1,0]) * T for elem in V[:,0]]
    # convert velcity data in m/s to flux in cm**3/s
    q = [elem*100*Ru**2*np.pi/Q for elem in V[:,1]]
    return interp1d(t, q, kind='linear', bounds_error=False, fill_value=q[0])

q_in = mca_inlet(Ru[0], Q, T) # flow rate at the inlet
out_bc = '3wk' # specify type of outlet bc as 3-element WK
out_args = np.array([14130/RES, 7200/RES, 2.4752e-3/COM]) # 3WK resistance and compliance

Solve the system (takes some time...)

In [5]:
an.solve(q_in, out_bc, out_args)

Progress: |██████████████████████████████████████████████████| 100.0% Complete


Redimensionalise and store the results

In [6]:
an.redimensionalise(R, Q)
an.dump_results('mca', './mca_vampy')

### Plot the results

The results are plotted using the plotting library delivered with VaMpy and Matplotlib:

In [7]:
from vampy import vamplot
import matplotlib.pylab as plt
from scipy.interpolate import interp2d

The following code is just going to make the plots look more visually appealing

In [8]:
plt.rcParams['axes.labelsize'] = 9
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['legend.fontsize'] = 9
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.serif'] = ['Arial']

WIDTH = 510
FACTOR = 1.0  # the fraction of the width you'd like the figure to occupy
fig_width_pt  = WIDTH * FACTOR
inches_per_pt = 1.0 / 72.27
golden_ratio  = (np.sqrt(5) - 1.0) / 2.0  # because it looks good
fig_width_in  = fig_width_pt * inches_per_pt  # figure width in inches
fig_height_in = fig_width_in * golden_ratio   # figure height in inches
fig_dims    = [fig_width_in, fig_height_in] # fig dims as a list

The pressure and velocity data is obtained from the files output during the previous step

In [9]:
# Load data files
P = np.loadtxt('./mca_vampy/mca/p0_mca.csv', delimiter=',')
U = np.loadtxt('./mca_vampy/mca/u0_mca.csv', delimiter=',')
# Define dimensional parameters
T = T * R**3/Q
tf = T*tc
L = Ru[0] * Lam[0] * R
# create arrays for time and space
t = np.linspace(tf-T, tf, P.shape[1])
x = np.linspace(0,L,P.shape[0])

To be able to plot the data, space and time are required to have the same dimension, so we need to interpolate

In [10]:
f = interp2d(t, x, P, kind='linear')
g = interp2d(t, x, U, kind='linear')
# redefine space to have the same length as time
x = np.linspace(0, L, len(t))
# calculate data on new grid
P = f(t, x)
U = g(t, x)

Now create the plots

In [12]:
vamplot.p3d_plot(fig_dims, t, P, L, 0, 'mca', './mca_vampy/')
vamplot.q3d_plot(fig_dims, t, U, L, 0, 'mca', './mca_vampy/')

The images are stored in the specifed folder

![p](mca_vampy/mca/mca_p3d0.png)
![q](mca_vampy/mca/mca_q3d0.png)

## References

[1] Diem AK, MacGregor Sharp M, Gatherer M, Bressloff NW, Carare RO and Richardson G (2017) Arterial Pulsations cannot Drive Intramural Periarterial Drainage: Significance for Aβ Drainage. Frontiers in Neuroscience 11:475. doi: https://doi.org/10.3389/fnins.2017.00475

[2] Diem AK and Bressloff NW (2017) VaMpy: A Python Package to Solve 1D Blood Flow Problems. Journal of Open Research Software 5(1), p.17. doi: http://doi.org/10.5334/jors.159

[3] Olufsen MS, Peskin CS, Kim WY, Pedersen EM, Nadim A and Larsen J (2000) Numerical Simulation and Experimental Validation of Blood Flow in Arteries with Structured-Tree Outflow Condition. Annals of Biomedical Engineering 28(11): 1281–1299, doi: https://doi.org/10.1114/1.1326031